## Write in Parquet file format using Trigger Once
Let us go through the details about reading and writing data to target location leveraging Spark Structured Streaming APIs.
* Let us write this data to target location using `parquet` file format. However, we will use `trigger(once=True)` to run only once.
* We will also validate by running `hdfs dfs -ls` command to see if the parquet files are copied or not.
* The files that are available at source at this time will be picked up automatically.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config('spark.sql.warehouse.dir', f'/user/{username}/warehouse'). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Incremental Loads using Spark Structured Streaming'). \
    master('yarn'). \
    getOrCreate()

In [2]:
spark.conf.set('spark.sql.shuffle.partitions', '16')

In [3]:
spark.conf.set('spark.sql.streaming.schemaInference', 'true')

In [4]:
ghactivity_df = spark. \
    readStream. \
    format('json'). \
    option('cleanSource', 'delete'). \
    load(f'/user/{username}/github/streaming/landing/ghactivity/')

In [5]:
from pyspark.sql.functions import year, month, dayofmonth, lpad

In [7]:
ghactivity_df = ghactivity_df. \
    withColumn('created_year', year('created_at')). \
    withColumn('created_month', lpad(month('created_at'), 2, '0')). \
    withColumn('created_dayofmonth', lpad(dayofmonth('created_at'), 2, '0'))

In [8]:
type(ghactivity_df)

pyspark.sql.dataframe.DataFrame

In [9]:
ghactivity_df.isStreaming

True

In [10]:
ghactivity_df. \
    writeStream. \
    partitionBy('created_year', 'created_month', 'created_dayofmonth'). \
    format('parquet'). \
    option("checkpointLocation", f"/user/{username}/github/streaming/bronze/checkpoint/ghactivity"). \
    option("path", f"/user/{username}/github/streaming/bronze/data/ghactivity"). \
    trigger(once=True). \
    start()

* Validating the checkpoint location. We can see multiple folders. These folders will have all the files that are required for the overhead of the checkpoint.

In [11]:
!hdfs dfs -ls /user/${USER}/github/streaming/bronze/checkpoint/ghactivity

Found 4 items
drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:23 /user/itv007304/github/streaming/bronze/checkpoint/ghactivity/commits
-rw-r--r--   3 itv007304 supergroup         45 2023-07-14 12:17 /user/itv007304/github/streaming/bronze/checkpoint/ghactivity/metadata
drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:17 /user/itv007304/github/streaming/bronze/checkpoint/ghactivity/offsets
drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:17 /user/itv007304/github/streaming/bronze/checkpoint/ghactivity/sources


In [13]:
!hdfs dfs -ls -R /user/${USER}/github/streaming/bronze/checkpoint/ghactivity/sources

drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:17 /user/itv007304/github/streaming/bronze/checkpoint/ghactivity/sources/0
-rw-r--r--   3 itv007304 supergroup       4432 2023-07-14 12:17 /user/itv007304/github/streaming/bronze/checkpoint/ghactivity/sources/0/0


In [14]:
!hdfs dfs -cat /user/${USER}/github/streaming/bronze/checkpoint/ghactivity/sources/0/0

v1
{"path":"hdfs://m01.itversity.com:9000/user/itv007304/github/streaming/landing/ghactivity/year=2023/month=07/dayofmonth=11/2023-07-11-0.json.gz","timestamp":1689199156255,"batchId":0}
{"path":"hdfs://m01.itversity.com:9000/user/itv007304/github/streaming/landing/ghactivity/year=2023/month=07/dayofmonth=11/2023-07-11-1.json.gz","timestamp":1689276160654,"batchId":0}
{"path":"hdfs://m01.itversity.com:9000/user/itv007304/github/streaming/landing/ghactivity/year=2023/month=07/dayofmonth=11/2023-07-11-2.json.gz","timestamp":1689276171657,"batchId":0}
{"path":"hdfs://m01.itversity.com:9000/user/itv007304/github/streaming/landing/ghactivity/year=2023/month=07/dayofmonth=11/2023-07-11-3.json.gz","timestamp":1689276182338,"batchId":0}
{"path":"hdfs://m01.itversity.com:9000/user/itv007304/github/streaming/landing/ghactivity/year=2023/month=07/dayofmonth=11/2023-07-11-4.json.gz","timestamp":1689276191737,"batchId":0}
{"path":"hdfs://m01.itversity.com:9000/user/itv007304/github/streaming/landin

In [16]:
!hdfs dfs -ls /user/${USER}/github/streaming/bronze/checkpoint/ghactivity/offsets

Found 1 items
-rw-r--r--   3 itv007304 supergroup        471 2023-07-14 12:17 /user/itv007304/github/streaming/bronze/checkpoint/ghactivity/offsets/0


In [17]:
!hdfs dfs -cat /user/${USER}/github/streaming/bronze/checkpoint/ghactivity/offsets/0

v1
{"batchWatermarkMs":0,"batchTimestampMs":1689351454553,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.join.stateFormatVersion":"2","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"16"}}
{"logOffset":0}

* Validating the data location. We should see the files in this location as we are just copying the files in the parquet file format.

In [18]:
!hdfs dfs -ls /user/${USER}/github/streaming/bronze/data/ghactivity

Found 2 items
drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:23 /user/itv007304/github/streaming/bronze/data/ghactivity/_spark_metadata
drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:17 /user/itv007304/github/streaming/bronze/data/ghactivity/created_year=2023


In [19]:
!hdfs dfs -ls -R /user/${USER}/github/streaming/bronze/data/ghactivity

drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:23 /user/itv007304/github/streaming/bronze/data/ghactivity/_spark_metadata
-rw-r--r--   3 itv007304 supergroup       8234 2023-07-14 12:23 /user/itv007304/github/streaming/bronze/data/ghactivity/_spark_metadata/0
drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:17 /user/itv007304/github/streaming/bronze/data/ghactivity/created_year=2023
drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:17 /user/itv007304/github/streaming/bronze/data/ghactivity/created_year=2023/created_month=07
drwxr-xr-x   - itv007304 supergroup          0 2023-07-14 12:22 /user/itv007304/github/streaming/bronze/data/ghactivity/created_year=2023/created_month=07/created_dayofmonth=11
-rw-r--r--   3 itv007304 supergroup  244196779 2023-07-14 12:19 /user/itv007304/github/streaming/bronze/data/ghactivity/created_year=2023/created_month=07/created_dayofmonth=11/part-00000-bdc32318-6047-45b2-a655-5b7e213b90b3.c000.snappy.parquet
-rw-r--r-- 

* Validating the source location to see if the files are delted or not.

In [20]:
!hdfs dfs -ls -R /user/${USER}/github/streaming/landing/ghactivity

drwxr-xr-x   - itv007304 supergroup          0 2023-07-12 17:57 /user/itv007304/github/streaming/landing/ghactivity/year=2023
drwxr-xr-x   - itv007304 supergroup          0 2023-07-12 17:57 /user/itv007304/github/streaming/landing/ghactivity/year=2023/month=07
drwxr-xr-x   - itv007304 supergroup          0 2023-07-13 15:27 /user/itv007304/github/streaming/landing/ghactivity/year=2023/month=07/dayofmonth=11
-rw-r--r--   3 itv007304 supergroup   96670341 2023-07-12 17:59 /user/itv007304/github/streaming/landing/ghactivity/year=2023/month=07/dayofmonth=11/2023-07-11-0.json.gz
-rw-r--r--   3 itv007304 supergroup   90660972 2023-07-13 15:22 /user/itv007304/github/streaming/landing/ghactivity/year=2023/month=07/dayofmonth=11/2023-07-11-1.json.gz
-rw-r--r--   3 itv007304 supergroup  104839007 2023-07-13 15:24 /user/itv007304/github/streaming/landing/ghactivity/year=2023/month=07/dayofmonth=11/2023-07-11-10.json.gz
-rw-r--r--   3 itv007304 supergroup  110194573 2023-07-13 15:24 /user/itv007304